In [1]:
import sys
import pandas
from pathlib import Path

sys.path.insert(0, str(Path('../').resolve()))
import cfe_realization_generator
import troute_generator

In [2]:
# INPUT DATA
domain_path = Path("../test-data/test-medium/domain")
forcing_path = Path("../test-data/test-medium/forcing")

In [3]:
# Clean existing dir if it already exists
!rm -rf data

In [4]:
# prepare directory for output data

# Note this directory structure mimicks that of the 
# NGIAB so that paths will be generated correctly.

output_dir = Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

config_dir = output_dir / 'config'
config_dir.mkdir(exist_ok=True)

domain_dir = output_dir / 'domain'
domain_dir.mkdir(exist_ok=True)

forcing_dir = output_dir / 'forcing'
forcing_dir.mkdir(exist_ok=True)

results_dir = output_dir / 'results'
results_dir.mkdir(exist_ok=True)


# populate domain and forcing data before building configs
!cp {str(domain_path)}/* {str(domain_dir)}
!cp {str(forcing_path)}/* {str(forcing_dir)}



In [5]:
# determine start and end dates from the forcing data
df = pandas.read_csv(list(forcing_dir.glob('*.csv'))[0])
st = pandas.to_datetime(df.time.min())

# set the end time 
#et = pandas.to_datetime(df.time.max())
from datetime import timedelta, datetime
et = st + timedelta(days=10) # using 10 days to keep simulations short for now.

## Generate T-Route Config

In [13]:
tconf = troute_generator.create_troute_configuration(simulation_start = st.to_pydatetime(),
                                                     simulation_end   = et.to_pydatetime(),
                                                     timestep_in_seconds = 300,
                                                     geopackage_path = Path('/ngen/data/domain/wb-2917533_upstream_subset.gpkg'),
                                                     input_data_path = Path('/ngen/data/results'),
                                                     output_file = config_dir/"ngen.yaml")

In [14]:
# need to fix some errors 

"""
network_topology_parameters -> supernetwork_parameters -> columns -> mainstem
  extra fields not permitted (type=value_error.extra)
"""

# network topology parameters are not set correctly when using the T-Route 
# pydantic classes. Load the class and manually remove "mainstem" from
# the columns dictionary.
_ = tconf.network_topology_parameters.supernetwork_parameters.columns.pop('mainstem')

# """
# compute_parameters -> restart_parameters -> start_datetime
#   datetime field must be specified as `datetime.datetime` object or string with
#   format ('%Y-%m-%d_%H:%M', '%Y-%m-%d_%H:%M:%S', '%Y-%m-%d %H:%M',
#   '%Y-%m-%d %H:%M:%S', '%Y/%m/%d %H:%M', '%Y/%m/%d %H:%M:%S') (type=value_error)
# """
dt = tconf.compute_parameters.restart_parameters.start_datetime 
tconf.compute_parameters.restart_parameters.start_datetime  = datetime.strftime(dt, '%Y-%m-%d %H:%M:%S')


In [15]:
from pydantic_yaml import to_yaml_str

with open(config_dir/"ngen.yaml", "w") as f:
    f.write(to_yaml_str(tconf, by_alias=True, exclude_none=True, sort_keys=False))

In [9]:
data_assimilation_parameters = tconf.compute_parameters.dict().get('data_assimilation_parameters',{})

## Generate CFE Config

In [11]:
cfe_realization_generator.create_global_cfe_realization(domain_dir/"cfe_noahowp_attributes.csv",
                                                         st,
                                                         et,
                                                         forcing_dir,
                                                         config_dir)


INFO:cfe_realization_generator:writing catchment config files (*.ini)
DEBUG:cfe_realization_generator:Creating global realization
INFO:cfe_realization_generator:Created global realization
DEBUG:cfe_realization_generator:create_default_cfe_realization completed successfully
DEBUG:cfe_realization_generator:CFE Init Configs: data/config/[cat-id]_config.ini
DEBUG:cfe_realization_generator:Realization JSON: data/config/realization.json


True

In [12]:
# DEBUGGING: check that the number of configs matches the number of forcing files
print(f'Number of Forcing Files: {len(list(forcing_dir.glob("*.csv")))}')
print(f'Number of Config Files : {len(list(config_dir.glob("*.ini")))}')

Number of Forcing Files: 250
Number of Config Files : 250
